In [0]:
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [0]:
np.random.seed(1337)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#               x=np.load('/content/drive/My Drive/X_arr.npy')
#y=np.load('/content/drive/My Drive/Minor-Major/DR/y_arr.npy')
#x=np.load('/content/drive/My Drive/Minor-Major/DR/X_arr.npy')

x_train=np.load('/content/drive/My Drive/AD_data/X_train.npy')
y_train=np.load('/content/drive/My Drive/AD_data/y_train.npy')
x_test=np.load('/content/drive/My Drive/AD_data/X_test.npy')
y_test=np.load('/content/drive/My Drive/AD_data/y_test.npy')
print(x_train.shape)
print(y_test.shape)
print(x_test.shape)
print(y_train.shape)



(2240, 224, 224, 3)
(960,)
(960, 224, 224, 3)
(2240,)


In [0]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4"
batch_size=32
nb_classes=4
nb_epoch=50
img_height=224
img_width=224
channels=3
nb_filters=32
kernel_size=(8,8)
INIT_LR=1e-3



In [0]:
from sklearn.utils import class_weight
print("Computing Class Weights")
weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

  

Computing Class Weights


In [0]:
valu = np.unique(y_test)
valu

array([0, 1, 2, 3])

In [0]:
x_train=x_train.reshape(x_train.shape[0],img_height,img_width,channels)
x_test=x_test.reshape(x_test.shape[0],img_height,img_width,channels)


In [0]:
input_shape=(img_height,img_width,channels)

In [0]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')


In [0]:
x_train=x_train/255
x_test=x_test/255

In [0]:
y_train=np_utils.to_categorical(y_train,nb_classes)
y_test=np_utils.to_categorical(y_test,nb_classes)

In [0]:
y_test.shape

(960, 4)

In [0]:
#keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)


In [0]:
# Let's try the VGG16 model
from keras.applications import resnet
from keras.applications import densenet
#pretrained_model = applications.VGG16(include_top=False, weights='imagenet')
from keras.applications.vgg19 import VGG19
import keras.applications
from keras.applications import xception
#base_model = VGG16(include_top=False, weights='imagenet')
#base_model = xception.Xception(include_top = False, weights = 'imagenet', input_tensor = None, input_shape = input_shape, pooling= None, classes = 5)
#base_model = VGG19(include_top = False, weights ='imagenet', input_tensor = None, input_shape = input_shape, pooling= None, classes = 4)
base_model = resnet.ResNet152(weights='imagenet', include_top=False, input_shape=(img_height, img_width, channels))

#keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
#base_model = densenet.DenseNet121(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_height, img_width, channels), pooling=None, classes=5)


234700800/234698864 [==============================] - 4s 0us/step


In [0]:
#base_model = applications.resnet.ResNet101(weights='imagenet', include_top=False, input_shape=(img_height, img_width, channels))
#base_model=applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=input_shape, pooling=None, classes=21)
#base_model=applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=input_shape, pooling=None, classes=21)
#base_model=applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=input_shape, pooling=None, classes=21)

In [0]:
print(base_model.output_shape[1:])

(7, 7, 2048)


In [0]:
from keras.models import Sequential, Model, load_model
from keras import optimizers
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(output_dim=nb_classes, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=4)`
  


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

batch_size = 32
epochs = 40

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)
#es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.03, patience = 4, restore_best_weights = True)

history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[ModelCheckpoint('VGG19-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Epoch 1/40
70/70 [==============================] - 109s 2s/step - loss: 1.1450 - accuracy: 0.4888 - val_loss: 1.5155 - val_accuracy: 0.2646
Epoch 2/40


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


70/70 [==============================] - 60s 858ms/step - loss: 0.7457 - accuracy: 0.6701 - val_loss: 2.0484 - val_accuracy: 0.2531
Epoch 3/40
70/70 [==============================] - 60s 858ms/step - loss: 0.6370 - accuracy: 0.7143 - val_loss: 3.0458 - val_accuracy: 0.2531
Epoch 4/40
70/70 [==============================] - 60s 857ms/step - loss: 0.5995 - accuracy: 0.7388 - val_loss: 6.5557 - val_accuracy: 0.2531
Epoch 5/40
70/70 [==============================] - 60s 857ms/step - loss: 0.5451 - accuracy: 0.7509 - val_loss: 10.8484 - val_accuracy: 0.2531
Epoch 6/40
70/70 [==============================] - 60s 857ms/step - loss: 0.5116 - accuracy: 0.7598 - val_loss: 12.0382 - val_accuracy: 0.2531
Epoch 7/40
70/70 [==============================] - 60s 857ms/step - loss: 0.4750 - accuracy: 0.7884 - val_loss: 11.8994 - val_accuracy: 0.2531
Epoch 8/40
70/70 [==============================] - 60s 857ms/step - loss: 0.4610 - accuracy: 0.7964 - val_loss: 12.0084 - val_accuracy: 0.2531
Epoch 

In [0]:
y_pred=base_model.predict(x_test)

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4778883432348569
Test accuracy: 0.878125011920929


In [0]:
y_pred = model.predict(x_test)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

y_pred = [np.argmax(y) for y in y_pred]
y_test = [np.argmax(y) for y in y_test]

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')


print("Precision: ", precision)
print("Recall: ", recall)

Test score: 0.4778883432348569
Test accuracy: 0.878125011920929
Precision:  0.8815448768338832
Recall:  0.878125


In [0]:
print(y_pred)

[0, 1, 0, 0, 1, 1, 1, 1, 1, 3, 2, 2, 1, 2, 2, 0, 3, 3, 2, 0, 2, 2, 0, 1, 3, 2, 1, 3, 1, 3, 1, 0, 0, 1, 1, 1, 0, 3, 0, 2, 2, 1, 3, 0, 2, 3, 2, 0, 2, 0, 3, 0, 2, 0, 1, 1, 3, 1, 1, 0, 2, 2, 2, 2, 0, 0, 0, 3, 1, 0, 1, 2, 1, 3, 3, 0, 2, 0, 3, 2, 0, 2, 2, 1, 3, 2, 0, 0, 1, 3, 0, 3, 1, 3, 0, 1, 0, 3, 0, 1, 1, 1, 3, 1, 0, 0, 3, 2, 1, 3, 2, 0, 0, 2, 0, 2, 1, 2, 2, 0, 0, 0, 2, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 1, 1, 1, 0, 0, 2, 3, 2, 1, 1, 3, 2, 3, 2, 2, 1, 3, 0, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 1, 2, 2, 0, 3, 0, 1, 0, 3, 1, 3, 1, 3, 2, 0, 1, 0, 3, 2, 0, 0, 0, 2, 3, 3, 3, 2, 0, 1, 2, 3, 0, 1, 3, 3, 3, 2, 0, 0, 0, 1, 1, 0, 0, 1, 3, 2, 0, 0, 0, 3, 3, 2, 1, 0, 3, 0, 2, 0, 1, 3, 1, 0, 3, 0, 2, 3, 1, 3, 0, 2, 1, 0, 1, 0, 0, 2, 0, 2, 3, 1, 1, 2, 2, 2, 2, 2, 1, 3, 2, 3, 0, 2, 0, 0, 3, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 2, 1, 3, 3, 2, 2, 3, 0, 3, 0, 0, 0, 2, 2, 1, 0, 3, 1, 1, 2, 3, 0, 0, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 0, 2, 0, 0, 0, 3, 1, 2, 0, 1, 3, 2, 2, 3, 1, 0, 2, 3, 1, 0, 3, 1, 2, 0, 1, 0, 2, 

In [0]:
model.save("drive/My Drive/AD_data/ResNet152CNN.h5")

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88       253
           1       0.92      1.00      0.96       216
           2       0.85      0.90      0.88       248
           3       0.92      0.70      0.80       243

    accuracy                           0.88       960
   macro avg       0.88      0.88      0.88       960
weighted avg       0.88      0.88      0.87       960



In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
results = confusion_matrix(y_test, y_pred)
print(results)

[[233   3  13   4]
 [  0 216   0   0]
 [  9   5 224  10]
 [ 36  12  25 170]]


In [0]:
results={}
results['vgg16_model_accuracy']=0.916
results['vgg16_model_precision']=0.90
results['vgg19_accuracy']=0.9416
results['vgg19_precision']=0.93
results['xception_model_accuracy']=0.89
results['xception_model_precision']=0.88
results['inception_v3_model_accuracy']=0.952
results['inception_v3_model_precision']=0.94
